<a href="https://colab.research.google.com/github/SergheiMihailov/ml-project-cassava/blob/main/ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
!gdown --id "1NdsNtg7RpTESOKojFbG5vlJMyxbbnNfx"

Downloading...
From: https://drive.google.com/uc?id=1NdsNtg7RpTESOKojFbG5vlJMyxbbnNfx
To: /content/train_images.zip
2.57GB [00:32, 79.9MB/s]


In [ ]:
import os
os.chdir('/content/')
!unzip -qq -o train_images.zip

In [ ]:
import pandas as pd
import json
data = pd.read_csv('train.csv')
f = open('label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}
data['class_name'] = data.label.map(real_labels)

In [ ]:
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

train_path = 'train_images/'
train,val = train_test_split(data, test_size = 0.10, random_state = 42, stratify = data['class_name'])

IMG_SIZE = 512
SIZE = (IMG_SIZE,IMG_SIZE)
CLASSES = 5
BATCH_SIZE = 16
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
dataget = ImageDataGenerator()

train_set = dataget.flow_from_dataframe(train,
                                directory = train_path,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                target_size = SIZE,
                                color_mode="rgb",
                                class_mode = 'categorical',
                                batch_size = BATCH_SIZE)

val_set = dataget.flow_from_dataframe(val,
                                directory = train_path,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                target_size = SIZE,
                                color_mode="rgb",
                                class_mode = 'categorical',
                                batch_size = BATCH_SIZE)

Found 19257 validated image filenames belonging to 5 classes.
Found 2140 validated image filenames belonging to 5 classes.


In [ ]:
mobileNetV3Small = tf.keras.applications.MobileNetV3Small(
    input_shape=INPUT_SHAPE, alpha=1, minimalistic=True, include_top=True,
    weights=None, input_tensor=None, classes=5, pooling='avg',
    dropout_rate=0, classifier_activation='softmax'
)
    
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
inputs = tf.keras.applications.mobilenet_v3.preprocess_input(inputs)

model = tf.keras.Model(inputs, mobileNetV3Small(inputs))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])


model.fit(
        train_set,
        steps_per_epoch=train_set.n // 32,
        epochs=30,
        validation_data=val_set,
        validation_steps=val_set.n // 32)

Epoch 1/30
601/601 [==============================] - 174s 280ms/step - loss: 0.3671 - accuracy: 0.6136 - val_loss: 0.3942 - val_accuracy: 0.6288
Epoch 2/30
601/601 [==============================] - 166s 277ms/step - loss: 0.3115 - accuracy: 0.6397 - val_loss: 0.4245 - val_accuracy: 0.6326
Epoch 3/30
601/601 [==============================] - 166s 275ms/step - loss: 0.2953 - accuracy: 0.6570 - val_loss: 0.5692 - val_accuracy: 0.1117
Epoch 4/30
601/601 [==============================] - 165s 275ms/step - loss: 0.2676 - accuracy: 0.6966 - val_loss: 0.7991 - val_accuracy: 0.1117
Epoch 5/30
601/601 [==============================] - 165s 275ms/step - loss: 0.2534 - accuracy: 0.7198 - val_loss: 0.6540 - val_accuracy: 0.1184
Epoch 6/30
601/601 [==============================] - 165s 275ms/step - loss: 0.2475 - accuracy: 0.7246 - val_loss: 0.8373 - val_accuracy: 0.1629
Epoch 7/30
601/601 [==============================] - ETA: 0s - loss: 0.2345 - accuracy: 0.7416